In [1]:
from glob import glob
from pandas import read_fwf
from bokeh.plotting import save, figure, output_file, show
from bokeh.models import ColumnDataSource, OpenURL, TapTool, BasicTickFormatter
from bokeh.models.tools import HoverTool
from bokeh.transform import factor_cmap
from bokeh.io import export_png

In [2]:
file_list = glob('data/mod_data/*.txt')
data_file = file_list[0]

'data/mod_data\\hatp23_oc_modelstats.txt'

In [3]:
output = "plots/ttv/" + data_file[14:-4] + ".html"
output_file(output)

end = data_file.find('_oc')
if data_file[14:18] == "hatp":
    name = "HAT-P-" + data_file[18:end]
elif data_file[14:18] == "wasp":
    name = "WASP-" + data_file[18:end]

In [4]:
ref = read_fwf(data_file, skiprows=1, header=None, sep="\t+")
ref = ref.drop(ref[ref[21] == 1].index)

df = ref.drop(ref[ref[8] == 'ThisStudy'].index)
this = ref.drop(ref[ref[8] != 'ThisStudy'].index)

In [5]:
source = ColumnDataSource(data=dict(
                epoch = df[4],
                res = df[6]*1440,
                LCType = df[8], 
                err_l = (df[6] - df[1])*1440,
                err_h = (df[6] + df[1])*1440,
        
                img_link = "../../" + df[22] + "preview.png",
                link = "../../" + df[22] + "preview.html"
            ))

In [6]:
sourceThis = ColumnDataSource(data=dict(
                epoch=this[4],
                res=this[6]*1440,
                LCType=this[8], 
                err_l = (this[6] - this[1])*1440,
                err_h = (this[6] + this[1])*1440,
    
                img_link = "../../" + this[22] + "lc.png",
                link = "../../" + this[23] + "preview.html"
            ))

In [7]:
p = figure(sizing_mode="stretch_both",                             
        tools="pan, box_zoom, save, reset, wheel_zoom, help", 
        title="TTV Diagram for " + name,
        x_axis_label = "Epoch (cycle)", 
        y_axis_label = "O-C (min)")

In [8]:
p.segment(x0=df[4],
        y0=(df[6] - df[1])*1440,
        x1=df[4],
        y1=(df[6] + df[1])*1440,
        color = 'black'
        )

p.segment(x0=df[4],
        y0=(df[6] - df[1])*1440,
        x1=df[4],
        y1=(df[6] + df[1])*1440,
        color = 'black'
        )

GlyphRenderer(id='1047', ...)

In [9]:
p.circle('epoch', 'res',                                         
                size=10,                                              
                source=source,                                        
                color=factor_cmap('LCType',
                                ['gray', 'turquoise', 'darkblue'],
                                ['Database', 'Literature', 'TESS']),
                legend_group = 'LCType', 
                name = "needsHover"
            )

p.circle('epoch', 'res',                     
                size=12,
                source=sourceThis,
                color=factor_cmap('LCType',
                                ['red'],
                                ['ThisStudy']),
                legend_group='LCType',
                name = "needsHover"
        )

GlyphRenderer(id='1080', ...)

In [10]:
p.legend.title = "Light Curve Type"
p.legend.location = "top_left" 

In [11]:
hover = HoverTool(names=['needsHover'])
hover.tooltips = """
          <div>
            <h3>{title} </h3>
            <div><strong>Click to see the details</strong></div>
            <div><img src="@img_link" alt="Sorry, the image didn't load properly." width="200" /></div>
          </div>
        """.format(title = "Light Curve Preview")
p.add_tools(hover)

In [12]:
url = "@link"
tap = TapTool(names = ["needsHover"]) 
tap.callback = OpenURL(url=url)
p.add_tools(tap)

In [13]:
p.xaxis.axis_label_text_font_size = "15pt"
p.yaxis.axis_label_text_font_size = "15pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.formatter = BasicTickFormatter(use_scientific=False)
p.yaxis.formatter = BasicTickFormatter(use_scientific=False)
p.title.text_font_size = '20pt'
p.legend.label_text_font_size = '13pt'
p.legend.title_text_font_size = '14pt'
    
#export_png(p, filename="plots/ttv_images/" + data_file[14:-4] + ".png") # exporting as png image to use in the homepage
save(p)

'C:\\Users\\muhay\\3D Objects\\v2\\plots\\ttv\\hatp23_oc_modelstats.html'